In [ ]:
import roboverse
env = roboverse.make('Widow250PickPlace-v1', observation_mode="pixels", gui=True)
env.reset()
for _ in range(1000):
    ext_obs, reward, done, infos = env.step(env.action_space.sample())
    print(ext_obs["state"][:3])
env.close()

In [ ]:
!python scripts/scripted_collect.py -n 100 -t 30 -e Widow250PickPlace-v0 -pl grasp -a grasp_success_target --noise=0.1 --gui

In [ ]:
!python scripts/scripted_collect.py -n 100 -t 30 -e Widow250PickPlace-v0 -pl pickplace -a place_success_target --noise=0.1 --gui

In [ ]:
!python scripts/scripted_collect.py -n 100 -t 30 -e Widow250PickPlaceMultiObject-v0 -pl pickplace -a place_success_target --noise=0.1 --gui

In [ ]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

import roboverse

env = gym.make("Pendulum-v1", render_mode="rgb_array")
env = roboverse.make("Widow250PickPlaceMultiObject-v0",
                         gui=True,
                         transpose_image=False)
env.reset()

# The noise objects for TD3
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = TD3("MultiInputPolicy", env, buffer_size=100000, action_noise=action_noise, verbose=1, learning_starts=0)
for i in range(500):
    print(i)
    action = env.action_space.sample()
    next_obs, reward, done, infos = env.step(action)
    model.replay_buffer.add(next_obs, next_obs, action, reward, done, [{}])
model.learn(total_timesteps=10000, log_interval=10)
model.save("data/td3")
vec_env = model.get_env()

del model # remove to demonstrate saving and loading

model = TD3.load("data/td3")

obs = vec_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

In [1]:
import gymnasium as gym
import panda_gym
import time


env = gym.make('PandaPickAndPlace-v3', render_mode="human")

observation, info = env.reset()

for _ in range(10000):
    action = env.action_space.sample() # random action
    observation, reward, terminated, truncated, info = env.step(action)
    print(reward)
    print(observation)
    time.sleep(1)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

pybullet build time: Oct 14 2023 15:44:17


argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=5
argv[0] = --unused
argv[1] = --background_color_red=0.8745098114013672
argv[2] = --background_color_green=0.21176470816135406
argv[3] = --background_color_blue=0.1764705926179886
argv[4] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (Intel(R) UHD Graphics 630)
GL_VERSION=4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.10
pthread_getconcurrency()=0
Version = 4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Microsoft Corporation
Renderer = D

In [1]:
! python roboverse/envs/widow250_eeposition.py

pybullet build time: Oct 14 2023 15:44:17
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (Intel(R) UHD Graphics 630)
GL_VERSION=4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.10
pthread_getconcurrency()=0
Version = 4.1 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Microsoft Corporation
Renderer = D3D12 (Intel(R) UHD Graphics 630)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Microsoft Corporation
ven = Microsoft Corporation
/home/enhupgu/miniconda3/envs/roboverse/li

In [ ]:
import gymnasium as gym
import numpy as np
import roboverse

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3 import DDPG, HerReplayBuffer
from sb3_contrib import TQC
from sb3_contrib.common.wrappers import TimeFeatureWrapper
from stable_baselines3.common.vec_env import VecNormalize
model = TD3()
model.replay_buffer.add(observation, next_observation, action, reward, done, [{}])

# Script

In [1]:
import time
import numpy as np
from roboverse.policies import policies
from stable_baselines3.common.utils import set_random_seed


def make_env(env_id: str, rank: int, seed: int = 0):
    """
    Utility function for multiprocessed env.

    :param env_id: the environment ID
    :param num_env: the number of environments you wish to have in subprocesses
    :param seed: the inital seed for RNG
    :param rank: index of the subprocess
    """
    def _init():
        env = roboverse.make(env_id,
                         gui=False,
                         observation_mode="pixels",
                         transpose_image=False)
        #env = TimeFeatureWrapper(env)
        #env.reset(seed=seed + rank)
        env.reset()
        return env
    set_random_seed(seed)
    return _init


def collect_data(env, model, policy, target, num_trajectories=100, num_timesteps=30):
    policy_class = policies[policy]
    policy = policy_class(env)
    num_success = 0
    num_saved = 0
    accept_trajectory_key = target
    noise = 0.1
    EPSILON = 0.1

    while num_saved < num_trajectories:
        num_saved += 1
        num_steps = 1e6
        rewards = []
        env.reset()
        policy.reset()
        time.sleep(0.1)
        for j in range(num_timesteps):
            action, agent_info = policy.get_action()

            # In case we need to pad actions by 1 for easier realNVP modelling 
            env_action_dim = env.action_space.shape[0]
            #if env_action_dim - action.shape[0] == 1:
            #    action = np.append(action, 0)
            action += np.random.normal(scale=noise, size=(env_action_dim,))
            action = np.clip(action, -1 + EPSILON, 1 - EPSILON)
            observation = env.get_observation()
            observation["image"] = np.transpose(observation["image"], (2, 0, 1))
            next_observation, reward, done, info = env.step(action)
            next_observation["image"] = np.transpose(next_observation["image"], (2, 0, 1))
            rewards.append(reward)
            model.replay_buffer.add(observation, next_observation, action, reward, np.array([done]), [{}])

            if info[accept_trajectory_key] and num_steps > 1e3:
                num_steps = j

            if info[accept_trajectory_key] and j > 23:
                break
            if done or agent_info['done']:
                break

        if info[accept_trajectory_key]:
            PRINT = False
            if PRINT:
                print("num_timesteps: ", num_steps, rewards)
                #print(observation["image"].shape)
                #print(next_observation["image"].shape)
            num_success += 1
        if num_saved%100 == 0:
            print(f"num_trajectories: {num_saved} success rate: {num_success/num_saved} Reward: {sum(rewards)}")

    print("success rate: {}".format(num_success / (num_saved)))


pybullet build time: Oct 14 2023 15:44:17


In [2]:
import gymnasium as gym
import numpy as np
import roboverse

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3 import DDPG, HerReplayBuffer
from sb3_contrib import TQC
from sb3_contrib.common.wrappers import TimeFeatureWrapper
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback


env = roboverse.make("Widow250PickPlace-v2",
                         gui=False,
                         observation_mode="pixels",
                         transpose_image=False)
#env = TimeFeatureWrapper(env)
#env = DummyVecEnv([make_env("Widow250PickPlace-v1", i) for i in range(4)])
obs = env.reset()

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path="./data/tqc/",
  name_prefix="tqc_model",
  save_replay_buffer=False,
  save_vecnormalize=False,
)

model = TQC(env=env, batch_size=2048, buffer_size=20000, gamma=0.95, learning_rate=0.001, policy='MultiInputPolicy',
             policy_kwargs=dict(net_arch=[512, 512, 512], n_critics=2),
             replay_buffer_class=HerReplayBuffer,
             replay_buffer_kwargs=dict(goal_selection_strategy='future', n_sampled_goal=4,), #max_episode_length=200,online_sampling=True,
             tau=0.05, learning_starts=200, verbose=1)

COLLECT=True
if COLLECT:
    collect_data(env, model, "pickplace", "place_success_target", 100, 30)
    model.save_replay_buffer(f"data/tqc_expert_pick_place")
else:
    print("load_replay_buffer")
    model.load_replay_buffer(f"data/tqc_expert_pick_place")

# print("start pre-training from buffer only")
# model.learn(total_timesteps=0, log_interval=5, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
# model.train(gradient_steps=10000)

print("start learning")
model.learn(total_timesteps=20000, callback=checkpoint_callback, log_interval=5, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
model.save("data/tqc")
model.save_replay_buffer(f"data/tqc_expert_pick_place_trained")

print("finish learning")

In [2]:
import gymnasium as gym
import numpy as np
import roboverse

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3 import DDPG, HerReplayBuffer
from sb3_contrib import TQC
from sb3_contrib.common.wrappers import TimeFeatureWrapper
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback


env = roboverse.make("Widow250PickPlace-v2",
                         gui=False,
                         observation_mode="pixels",
                         transpose_image=False)
#env = TimeFeatureWrapper(env)
#env = DummyVecEnv([make_env("Widow250PickPlace-v1", i) for i in range(4)])
obs = env.reset()

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path="./data/tqc/",
  name_prefix="tqc_model",
  save_replay_buffer=False,
  save_vecnormalize=False,
)

model = TQC(env=env, batch_size=2048, buffer_size=20000, gamma=0.95, learning_rate=0.001, policy='MultiInputPolicy',
             policy_kwargs=dict(net_arch=[512, 512, 512], n_critics=2),
             replay_buffer_class=HerReplayBuffer,
             replay_buffer_kwargs=dict(goal_selection_strategy='future', n_sampled_goal=4),
             tau=0.05, learning_starts=200, verbose=1)

# COLLECT=True
# if COLLECT:
#     collect_data(env, model, "grasp", "grasp_success_target", 5000, 30)
#     model.save_replay_buffer(f"data/tqc_expert_grasp")
# else:
#     print("load_replay_buffer")
#     model.load_replay_buffer(f"data/tqc_expert_grasp")

# print("start pre-training from buffer only")
# model.learn(total_timesteps=0, log_interval=5, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
# model.train(gradient_steps=10000)

print("start learning")
model.learn(total_timesteps=20000, callback=checkpoint_callback, log_interval=5, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
model.save("data/tqc")
model.save_replay_buffer(f"data/tqc_expert_grasp_trained")

print("finish learning")

/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


Output()

/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:219: 
DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is 
recommended to rewrite the environment with new step API. 
  logger.deprecation(

/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:225: 
DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):

...

start learning


/home/enhupgu/miniconda3/envs/roboverse/lib/python3.8/site-packages/gymnasium/core.py:311: UserWarning: WARN: 
env.compute_reward to get variables from other wrappers is deprecated and will be removed in v1.0, to get this 
variable you can do `env.unwrapped.compute_reward` for environment variables or 
`env.get_wrapper_attr('compute_reward')` that will search the reminding wrappers.
  logger.warn(

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99       |
|    ep_rew_mean     | -99      |
| time/              |          |
|    episodes        | 5        |
|    fps             | 10       |
|    time_elapsed    | 45       |
|    total_timesteps | 495      |
| train/             |          |
|    actor_loss      | -41.7    |
|    critic_loss     | 0.487    |
|    ent_coef        | 0.746    |
|    ent_coef_loss   | -3.95    |
|    learning_rate   | 0.001    |
|    n_updates       | 294      |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99       |
|    ep_rew_mean     | -99      |
| time/              |          |
|    episodes        | 10       |
|    fps             | 8        |
|    time_elapsed    | 112      |
|    total_timesteps | 990      |
| train/             |          |
|    actor_loss      | -46.6    |
|    critic_loss     | 0.408    |
|    ent_coef        | 0.455    |
|    ent_coef_loss   | -10.6    |
|    learning_rate   | 0.001    |
|    n_updates       | 789      |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99       |
|    ep_rew_mean     | -99      |
| time/              |          |
|    episodes        | 15       |
|    fps             | 8        |
|    time_elapsed    | 184      |
|    total_timesteps | 1485     |
| train/             |          |
|    actor_loss      | -33.1    |
|    critic_loss     | 0.276    |
|    ent_coef        | 0.277    |
|    ent_coef_loss   | -17.2    |
|    learning_rate   | 0.001    |
|    n_updates       | 1284     |
---------------------------------


In [ ]:
# del model # remove to demonstrate saving and loading
# model = TQC.load("data/tqc")

# start env with gui
env.close()
env = roboverse.make("Widow250PickPlace-v1",
                         gui=True,
                         observation_mode="pixels",
                         transpose_image=False)
model.set_env(env)
env = model.get_env()

obs = env.reset()
print("start render")
for i in range(int(1e4)):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render("human")
env.close()

In [2]:
params = OrderedDict([('batch_size', 2048),
             ('buffer_size', 1000000),
             #('env_wrapper', 'sb3_contrib.common.wrappers.TimeFeatureWrapper'),
             ('gamma', 0.95),
             ('learning_rate', 0.001),
             #('n_timesteps', 1000000.0),
             ('policy', 'MultiInputPolicy'),
             ('policy_kwargs', 'dict(net_arch=[512, 512, 512], n_critics=2)'),
             ('replay_buffer_class', 'HerReplayBuffer'),
             ('replay_buffer_kwargs',
              "dict( online_sampling=True, goal_selection_strategy='future', "
              'n_sampled_goal=4, )'),
             ('tau', 0.05),
             #('normalize', False)
             ])

In [6]:
import gymnasium as gym
import panda_gym
from stable_baselines3 import DDPG, HerReplayBuffer
from sb3_contrib import TQC
from sb3_contrib.common.wrappers import TimeFeatureWrapper
from stable_baselines3.common.vec_env import VecNormalize
from collections import OrderedDict


env = gym.make("PandaPickAndPlace-v3")
env = TimeFeatureWrapper(env)
model = TQC(env=env, batch_size=2048, buffer_size=100000, gamma=0.95, learning_rate=0.001, policy='MultiInputPolicy',
             policy_kwargs=dict(net_arch=[512, 512, 512], n_critics=2),
             replay_buffer_class=HerReplayBuffer,
             replay_buffer_kwargs=dict(goal_selection_strategy='future', n_sampled_goal=4),
             tau=0.05, log_interval=10, verbose=1)

model.learn(1_000_000)
model.save('data/pick_place/tqc')

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | -50      |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 60       |
|    time_elapsed    | 3        |
|    total_timesteps | 200      |
| train/             |          |
|    actor_loss      | -11.4    |
|    critic_loss     | 0.0321   |
|    ent_coef        | 0.907    |
|    ent_coef_loss   | -0.657   |
|    learning_rate   | 0.001    |
|    n_updates       | 99       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | -50      |
|  